In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import cv2

In [2]:
import os
import urllib.request
import zipfile

# Define dataset directory
dataset_dir = "../data/kitti_sample/"
zip_filename = "kitti_sample.zip"
zip_path = os.path.join(dataset_dir, zip_filename)
extracted_folder = os.path.join(dataset_dir, "training")  # KITTI usually has 'training' and 'testing' folders

# Create the dataset directory if it doesn't exist
os.makedirs(dataset_dir, exist_ok=True)

# Download if not already downloaded
if not os.path.exists(zip_path):
    print("Downloading KITTI sample dataset...")
    url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip"
    urllib.request.urlretrieve(url, zip_path)
    print("Download complete.")
else:
    print("Zip file already exists. Skipping download.")

# Extract if not already extracted
if not os.path.exists(extracted_folder):
    print("Extracting ZIP file...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)
    print("Extraction complete.")
else:
    print("Data already extracted. Skipping extraction.")




Zip file already exists. Skipping download.
Data already extracted. Skipping extraction.


In [3]:
import glob
import os

# Tiny dataset path
tiny_dataset_dir = "../data/kitti_tiny/training/image_2/"

# Find all images
tiny_image_paths = glob.glob(os.path.join(tiny_dataset_dir, "*.png"))
tiny_image_paths = sorted(tiny_image_paths)

print(f"Total images found in tiny dataset: {len(tiny_image_paths)}")


Total images found in tiny dataset: 100


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils.dataset import KITTITinyDataset
from models.depth_cnn import SimpleDepthCNN
from torch.utils.data import DataLoader

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
dataset = KITTITinyDataset(root_dir="../data/kitti_tiny/training/image_2/")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize model
model = SimpleDepthCNN().to(device)

# Define loss and optimizer
criterion = nn.L1Loss()  # Mean Absolute Error (good for depth)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs in dataloader:
        inputs = inputs.to(device)

        # Forward
        outputs = model(inputs)

        # Since we have no real depth maps, fake target = input's first channel (just for demo)
        targets = inputs[:, 0:1, :, :]  # Take R channel as dummy depth

        # Compute loss
        # Resize model output to match target
        outputs = torch.nn.functional.interpolate(outputs, size=targets.shape[-2:], mode="bilinear", align_corners=False)

        # Now compute loss
        loss = criterion(outputs, targets)


        # Backward + Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

print("Training complete!")


Using device: cpu
Epoch [1/5], Loss: 0.1963
Epoch [2/5], Loss: 0.1118
Epoch [3/5], Loss: 0.0934
Epoch [4/5], Loss: 0.0947
Epoch [5/5], Loss: 0.0787
Training complete!
